In [1]:
import numpy as np
import scipy as sp


In [2]:
def CarToSph(x, y, z):
    r = np.sqrt(x**2+y**2+z**2)
    theta = np.arccos(z/r)
    phi = np.arctan2(y, x)
    return r, theta, phi


def SphToCar(r, theta, phi):
    x = r*np.cos(phi)*np.sin(theta)
    y = r*np.sin(phi)*np.sin(theta)
    z = r*np.sin(theta)


In [3]:
N = 1000
particle = np.random.uniform(size=(N, 4))


In [72]:
level = int(np.ceil(np.log2(N)/3)) + 1
idx_particle = -np.ones(shape=(N, level, 3), dtype=np.int32)
for i in range(N):
    idx_particle[i, 0, :] = 0
    residu = particle[i, :3]
    for j in range(1, level):
        split = np.int32(residu > 0.5)
        idx_particle[i, j, :] = idx_particle[i, j-1, :]*2+split
        residu = 2*residu-split

In [35]:
def XYZToL(idx,n):
    return idx[2]*n*n+idx[1]*n+idx[0]

def LToXYZ(l,n):
    z = l//(n*n)
    y = (l-z*n*n)//n
    x = (l-z*n*n)%n
    return np.array([x,y,z])

In [38]:
LToXYZ(XYZToL([541,342,247],2**10),2**10)

array([541, 342, 247])

In [73]:
tree_idx = [[[] for j in range(8**i)] for i in range(level)]
for i in range(level):
    for j in range(N):
        tree_idx[i][XYZToL(idx_particle[j,i,:],2**i)].append(j)
    

In [81]:
def CellCenter(idx, level):
    return idx*2**(-level)+2**(-level-1)


In [82]:
def NeighboursRange(center_idx, center_level):
    return [np.maximum(0, center_idx[center_level, :]-1), np.minimum(2**center_level-1, center_idx[center_level, :]+1)]


def NeighboursChildRange(center_idx, center_level):
    return [np.maximum(0, 2*(center_idx[center_level, :]-1)), np.minimum(2**(center_level+1)-1, 2*center_idx[center_level, :]+3)]


In [78]:
idx_particle[0, 2, :], idx_particle[0, 3, :]


(array([2, 3, 2]), array([5, 7, 4]))

In [83]:
NeighboursRange(idx_particle[0, :, :], 2)

[array([1, 2, 1]), array([3, 3, 3])]

In [84]:
NeighboursChildRange(idx_particle[0, :, :], 2)

[array([2, 4, 2]), array([7, 7, 7])]

In [85]:
NeighboursRange(idx_particle[0, :, :], 3)

[array([4, 6, 3]), array([6, 7, 5])]

In [84]:
def InteractionList(cell_idx, N_level):
    for i in range(N_level):
        incluide = NeighboursChildRange(cell_idx, i-1) # Range of parent's neighbours
        exclude = NeighboursRange(cell_idx, i) # Range of neighbours
